Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

In [ ]:
%matplotlib inline

import io
import requests as rq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)

In [ ]:
host = 'climate.weather.gc.ca'
page = '/climate_data/bulk_data_e.html'
query = {'format': 'csv',
         'stationID': '51157',
         'Year': '2017',
         'Month': '4',
         'timeframe': '1',
         'submit': 'Download Data'}

In [ ]:
r = rq.get("http://"+host+page, params=query)

In [ ]:
r.status_code

In [ ]:
r.encoding = 'latin1'

In [ ]:
r.url

In [ ]:
r.headers

In [ ]:
f = io.BytesIO(r.content)

In [ ]:
df2 = pd.read_csv(f, skiprows=16, index_col='Date/Time', parse_dates=True)

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
gen_query = lambda year, month, station: {'format': 'csv',
                                 'stationID': str(station),
                                 'Year': str(year),
                                 'Month': str(month),
                                 'timeframe': '1',
                                 'submit': 'Download Data'}
gen_query(2017, 4, 51157)

In [ ]:
def get_month_data(year, month, station=51157):
    q = gen_query(year, month, station)
    host = 'climate.weather.gc.ca'
    page = '/climate_data/bulk_data_e.html'
    r = rq.get("http://"+host+page, params=q)
    if r.status_code != 200:
        return None
    r.encoding = 'latin1'
    f = io.BytesIO(r.content)
    df = pd.read_csv(f, skiprows=16, index_col='Date/Time', parse_dates=True)
    df = df.dropna(axis=1)
    df = df.drop(['Year', 'Month', 'Day', 'Time'], axis=1)
    try:
        df = df.drop(['Data Quality'], axis=1)
    except:
        pass
    return df

In [ ]:
df = get_month_data(2012, 3, 5415)

In [ ]:
df.shape

In [ ]:
df.head()


In [ ]:
df.info()

In [ ]:
temp_colname = df.columns[0]

In [ ]:
df[temp_colname].plot()

In [ ]:
temperatures = df[[temp_colname]]

In [ ]:
temperatures['Hour'] = df.index.hour

In [ ]:
temperatures

In [ ]:
hourgroup = temperatures.groupby('Hour')

In [ ]:
agg = hourgroup.aggregate(np.median)

In [ ]:
agg

In [ ]:
agg.plot()

In [ ]:
data_by_month = [get_month_data(2012, i, 5415) for i in range(1,13)]

In [ ]:
weather_2012 = pd.concat(data_by_month)

In [ ]:
weather_2012.info()

In [ ]:
tempcol = weather_2012.columns[0]
try:
    weather_2012[tempcol].plot()
except:
    pass